In [30]:
from obspy import read
import matplotlib.pyplot as plt
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from obspy.core import AttribDict
from obspy.io.sac import SACTrace
import numpy as np
import matplotlib.pylab as plt
import os
import pandas as pd
import shutil
from tqdm import tqdm
import warnings
from obspy import read_inventory, UTCDateTime
from obspy.core.inventory import Inventory, Network, Station, Channel, Site

In [146]:
network = "7B" # YO ENAM; ZA NoMelt
tstart = "2001-12-02T00:00:00"
tend = "2002-11-25T23:59:59"

comps = ['BHZ'] #['BHZ','BH1','BH2','BDH'] #["HXZ", "HX1", "HX2"] #["LHZ", "LH1", "LH2"] #["HHZ", "HH1", "HH2"] #["HHZ", "HH1", "HH2", "BDH"]
outunits = 'DISP' # DISP, VEL, ACC [For pressure channels, should use "VEL"] 
client = Client('IRIS')
stations = "*"
# If you want to specify the stations
#stations = ('A01','A03','A05', 'A07', 'A09', 'A11', 'A13', 'B02', 'B03', 'B06', 'B08', 'B10', 'B12', 'B13', 'C02', 'C04', 'C06', 'C12', 'C13', 'C15', 'F01')
#stations = ','.join(stations)

t1 = UTCDateTime(tstart)
t2 = UTCDateTime(tend)

# STATIONS
inventory = client.get_stations(network=network, station=stations,channel=comps[0], level='response')#starttime=t1, endtime=t2, 
print(inventory)


Inventory created at 2023-07-13T20:44:16.726800Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.52
		    http://service.iris.edu/fdsnws/station/1/query?network=7B&station=%...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (2):
			7B (2x)
		Stations (67):
			7B.MM01 (SIO OBS ID MM01)
			7B.MM02 (SIO OBS ID MM02)
			7B.MM03 (SIO OBS ID MM03)
			7B.MM04 (SIO OBS ID MM04)
			7B.MM05 (SIO OBS ID MM05)
			7B.MM06 (SIO OBS ID MM06)
			7B.MM07 (SIO OBS ID MM07)
			7B.MM08 (SIO OBS ID MM08)
			7B.MM09 (SIO OBS ID MM09)
			7B.MM10 (SIO OBS ID MM10)
			7B.MM11 (SIO OBS ID MM11)
			7B.MM12 (SIO OBS ID MM12)
			7B.NM01 (SIO OBS ID NM01)
			7B.NM02 (SIO OBS ID NM02)
			7B.NM03 (SIO OBS ID NM03)
			7B.NM04 (SIO OBS ID NM04)
			7B.NM05 (SIO OBS ID NM05)
			7B.NN01 (SIO OBS ID NN01)
			7B.NN02 (SIO OBS ID NN02)
			7B.NN03 (SIO OBS ID NN03)
			7B.NN04 (SIO OBS ID NN04)
			7B.SA01 (QUEENSLAND, AUSTRALIA)
			7B.SA02 (QUEENSLAND, AUSTRALIA)
			7B.SA03 (QUEENSLAND, AUSTRALIA)

In [147]:
#remove response for all downloaded files in a network

#the folder where all downloaded files in a network located
folder_path = '/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/output/7B/7B'

#the folder that store all files in a network after removing response. You need to create this folder first.
output_folder = '/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/7B'

sac_files = []

# loop through all station folders in the network folder
for station_folder in os.listdir(folder_path):
    station_folder_path = os.path.join(folder_path, station_folder)
    for root, dirs, files in os.walk(station_folder_path):
        # loop through all files in the station folder
        for file in files:
            if file.endswith(".sac"):
                # add file to sac_files list
                sac_files.append(os.path.join(root, file))

# count files that need to remove response
count = len(sac_files)
print(count)

#progress bar
pbar = tqdm(total=len(sac_files))

# list to store files with errors
abnormal_sac_files = []

# Load all SAC files in sac_files list using ObsPy
for sac_file in sac_files:
    try:
        st = read(sac_file)
        st.attach_response(inventory)
        st.remove_response(inventory=inventory)

        # Create output station folders
        relative_path = os.path.relpath(os.path.dirname(sac_file), folder_path)
        output_subfolder = os.path.join(output_folder, relative_path)
        os.makedirs(output_subfolder, exist_ok=True)

        # Generate output file path
        # Output files will have the same names as the original files
        output_file_name = os.path.basename(sac_file)
        output_file_path = os.path.join(output_subfolder, output_file_name)

        # Write the modified stream to the output file path
        st.write(output_file_path, format="SAC")
        pbar.update(1)
        
    #If there's an error when removing response, skip that file and continue            
    except ValueError as e:
        print(f"Skipping '{sac_file}': {str(e)}")
        abnormal_sac_files.append(sac_file)
        
        #If there's an error when removing response, copy the file as it is to new folder
        #error_file_path = os.path.join(output_subfolder, output_file_name)
        #shutil.copy(sac_file, error_file_path)
        #print(f"Original file copied to '{error_file_path}'")
        continue
                 
# Close the progress bar
pbar.close()

print("Files with errors:")
print(abnormal_sac_files)

8277


  1%|          | 78/8277 [00:17<31:47,  4.30it/s] WARNING: FIR normalized: sum[coef]=9.767192E-01;     


  3%|▎         | 234/8277 [00:52<29:37,  4.53it/s] WARNING: FIR normalized: sum[coef]=9.767192E-01;     


  5%|▍         | 390/8277 [01:27<31:26,  4.18it/s] WARNING: FIR normalized: sum[coef]=9.767192E-01;     


  7%|▋         | 546/8277 [02:03<30:33,  4.22it/s] WARNING: FIR normalized: sum[coef]=9.767192E-01;     


...

 93%|█████████▎| 7715/8277 [29:10<02:04,  4.50it/s] WARNING: FIR normalized: sum[coef]=9.767192E-01;     


 95%|█████████▌| 7869/8277 [29:45<01:34,  4.33it/s] WARNING: FIR normalized: sum[coef]=9.767192E-01;     


 97%|█████████▋| 8023/8277 [30:21<00:58,  4.32it/s] WARNING: FIR normalized: sum[coef]=9.767192E-01;     


 99%|█████████▉| 8177/8277 [30:57<00:25,  3.99it/s] WARNING: FIR normalized: sum[coef]=9.767192E-01;     


100%|█████████▉| 8275/8277 [31:20<00:00,  4.40it/s]


In [9]:
# Troubleshooting for abnormal_sac_files
# If you want to look at info of specific channel/station/network
network = "XE"
station = 'CC06'

# Download the specific response from an online source
client = Client("IRIS")
inventory_specific = client.get_stations(network=network,station=station,channel='BHZ', level="response")

# Channel info for reference
inventory_specific[0][0][0] #[0] or [0][0] for station and network info

Channel 'BHZ', Location '' 
	Time range: 2018-04-21T14:32:00.000000Z - 2019-05-21T02:41:00.000000Z
	Latitude: -5.8385, Longitude: -132.9518, Elevation: -4824.0 m, Local Depth: 0.0 m
	Azimuth: 0.00 degrees from north, clockwise
	Dip: -90.00 degrees down from horizontal
	Channel types: GEOPHYSICAL
	Sampling Rate: 50.00 Hz
	Sensor (Description): None (T240 Single-Sided/ABALONES)
	Response information available

In [67]:
# If you want to enter abnormal_sac_files by hand
abnormal_sac_files = ['/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/output/7B/7B/MM07/MM07.2019.329.00.00.00.BHZ.sac',
'/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/output/7B/7B/SM05/SM05.2019.328.00.00.00.BHZ.sac']

In [68]:
# See the date of the first file in abnormal_sac_files to enter 'time' below.
st = read(abnormal_sac_files[0],format="SAC")
print(st)

1 Trace(s) in Stream:
7B.MM07..BHZ | 2019-11-25T00:00:00.009581Z - 2019-11-25T23:59:59.009581Z | 1.0 Hz, 86400 samples

In [72]:
# Create an inventory for a abnormal sac file manually, and use this inventory to remove response
network = '7B'
station = 'SM05'
station_code = network+'.'+station+'..'+'BHZ'
time = "2019-11-26T00:00:00" # It is the best to use a dates close to the abnormal date, which are the most likely one to have response.

client = Client("IRIS")
inventory_specific = client.get_stations(network=network,station=station,channel='BHZ', level="response")
latitude = inventory_specific[0][0].latitude
longitude = inventory_specific[0][0].longitude
elevation = inventory_specific[0][0].elevation
depth = inventory_specific[0][0][0].depth
azimuth = inventory_specific[0][0][0].azimuth
dip = inventory_specific[0][0][0].dip
sample_rate = inventory_specific[0][0][0].sample_rate

# We'll first create all the various objects. These strongly follow the
# hierarchy of StationXML files.
inv = Inventory(
    # We'll add networks later.
    networks=[],
    # The source should be the id whoever create the file.
    source="URSeismo-OBS")

net = Network(
    # This is the network code according to the SEED standard.
    code=network,
    # A list of stations. We'll add one later.
    stations=[],
    description="manually_remove_response_inventory",
    # Start-and end dates are optional.
    #start_date=obspy.UTCDateTime(2016, 1, 2)
)

sta = Station(
    # This is the station code according to the SEED standard.
    code=station,
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    #creation_date=obspy.UTCDateTime(2016, 1, 2),
    site=Site(name=station))

cha = Channel(
    # This is the channel code according to the SEED standard.
    code="BHZ",
    # This is the location code according to the SEED standard.
    location_code="",
    # Note that these coordinates can differ from the station coordinates.
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    depth=depth,
    azimuth=azimuth,
    dip=dip,
    sample_rate=sample_rate)

# By default this accesses the NRL online. Offline copies of the NRL can
# also be used instead
nrl = client.get_stations(network=network,station=station,channel='BHZ', level="response")
# The contents of the NRL can be explored interactively in a Python prompt,
# see API documentation of NRL submodule:
# http://docs.obspy.org/packages/obspy.clients.nrl.html
# Here we assume that the end point of data logger and sensor are already
# known:
datetime = UTCDateTime(time)
response = nrl.get_response(station_code, datetime)


# Now tie it all together.
cha.response = response
sta.channels.append(cha)
net.stations.append(sta)
inv.networks.append(net)

# And finally write it to a StationXML file. We also force a validation against
# the StationXML schema to ensure it produces a valid StationXML file.
#
# Note that it is also possible to serialize to any of the other inventory
# output formats ObsPy supports.
inv.write("station.xml", format="stationxml", validate=True)


In [74]:
#the folder that store abnormal sac files in a network after removing response. You need to create this folder first.
output_folder = '/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/removed_response_output/7B/'+station
inventory = read_inventory("station.xml")
st.attach_response(inventory)
for file in abnormal_sac_files:
    try:
        # Read the SAC file
        st = read(file)

        # Remove instrument response
        st.attach_response(inventory)
        st.remove_response(inventory=inventory)
        
        print('Successfully removed response for the following abnormal sac file:')
        print(st)
        
        # Save the processed data 
        filename = os.path.basename(file)
        output_file = os.path.join(output_folder, filename)
        st.write(output_file, format='SAC')
        
        # Delete the file from the abnormal_sac_file list
        abnormal_sac_files.remove(file)
        
    except ValueError as e:
        print(f"Skipping '{file}': {str(e)}")
        continue

1 Trace(s) in Stream:
7B.SM05..BHZ | 2019-11-24T00:00:00.009556Z - 2019-11-24T23:59:59.009556Z | 1.0 Hz, 86400 samples


In [ ]:
# Test if a sac file is normal
# If it's normal it will print out info of the file, otherwise error 'No maching response' will occur
st = read('/gpfs/fs2/scratch/tolugboj_lab/Prj4c_OJP/2_Data/OJ/fetch_NOISE-master/output/YL/YL/C02/C02.2009.331.00.00.00.BHZ.sac', format="SAC")
st.attach_response(response)
print(st)